In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Load data

In [ ]:
from datasets import load_dataset, Audio

data_name = 'thinhlpg/vivoice-distil'
data = load_dataset(data_name, split='train[:20%]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/56761 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['channel', 'text', 'audio'],
    num_rows: 11352
})

In [ ]:
# resampling rate
data = data.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
example = data[2]
example

{'channel': '@HuynhDuyKhuongofficial',
 'text': 'Mình chỉ thích làm việc của mình thôi.',
 'audio': {'path': 'audio_545.68_546.6.wav',
  'array': array([-0.00692326, -0.01003117, -0.00852066, ...,  0.00077414,
          0.00070151,  0.00043786]),
  'sampling_rate': 16000}}

# Load model

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer

model_name = "openai/whisper-base"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="vietnamese", task="transcribe")
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="vietnamese", task="transcribe")


input_features = processor(example["audio"]["array"], sampling_rate=example['audio']["sampling_rate"], return_tensors="pt").input_features

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

In [ ]:
#generate token ids
predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
print("model generation:", transcription[0])
print('True text:', example['text'])

model generation:  Mình chỉ rất là bị của mình nói
True text: Mình chỉ thích làm việc của mình thôi.


prepare data

In [ ]:
audio = processor.feature_extractor(example['audio']["array"], sampling_rate=example['audio']["sampling_rate"], return_tensors="pt").input_features[0]
audio.shape

torch.Size([80, 3000])

In [ ]:
label = tokenizer(example['text'], return_tensors="pt")
label['input_ids'].shape

torch.Size([1, 16])

In [ ]:
def prepare_data(batch):
  audio = batch["audio"]

  # compute log-Mel input features from input audio array
  batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
  # encode target text to label ids
  batch["labels"] = tokenizer(batch["text"]).input_ids
  return batch

In [ ]:
example = data[0]
prepare_data(example)

{'channel': '@CuThongThai',
 'text': 'Nhìn lên sạng trừng khoán và nó giàu nhanh như biến đổi xen.',
 'audio': {'path': 'audio_85.86_88.22.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00065497,
         -0.00055135, -0.00058164]),
  'sampling_rate': 16000},
 'input_features': array([[-0.6517712, -0.6517712, -0.6517712, ..., -0.6517712, -0.6517712,
         -0.6517712],
        [-0.6517712, -0.6517712, -0.6517712, ..., -0.6517712, -0.6517712,
         -0.6517712],
        [-0.6517712, -0.6517712, -0.6517712, ..., -0.6517712, -0.6517712,
         -0.6517712],
        ...,
        [-0.6517712, -0.6517712, -0.6517712, ..., -0.6517712, -0.6517712,
         -0.6517712],
        [-0.6517712, -0.6517712, -0.6517712, ..., -0.6517712, -0.6517712,
         -0.6517712],
        [-0.6517712, -0.6517712, -0.6517712, ..., -0.6517712, -0.6517712,
         -0.6517712]], dtype=float32),
 'labels': [50258,
  50278,
  50359,
  50363,
  45,
  71,
  41320,
  33368,
  262,
  5890,
 

In [ ]:
data_prepared = data.map(prepare_data, remove_columns=data.column_names) # , num_proc=4
#data_prepared.set_format(type='torch', columns=['input_features', 'labels'], output_all_columns=True)

Map:   0%|          | 0/11352 [00:00<?, ? examples/s]

In [ ]:
data_prepared_split = data_prepared.train_test_split(test_size=0.2)
data_prepared_split

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 9081
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2271
    })
})

# Fine-tune model

In [ ]:
model.generation_config.language = "vietnamese"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
      # split inputs and labels since they have to be of different lengths and need different padding methods
      # first treat the audio inputs by simply returning torch tensors
      input_features = [{"input_features": feature["input_features"]} for feature in features]
      batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

      # get the tokenized label sequences
      label_features = [{"input_ids": feature["labels"]} for feature in features]
      # pad the labels to max length
      labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

      # replace padding with -100 to ignore loss correctly
      labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

      # if bos token is appended in previous tokenization step,
      # cut bos token here as it's append later anyways
      if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
          labels = labels[:, 1:]

      batch["labels"] = labels_batch
      return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
reduced_dataset = data_prepared_split['train'].select(range(20))
a = data_collator(reduced_dataset)

In [ ]:
a.keys()

dict_keys(['input_features', 'labels'])

load metrics WER

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

train arguments

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-base-datavie",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_prepared_split["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 9081
})

In [ ]:
print(type(data_prepared_split['train'][0]['input_features']))
print(type(data_prepared_split['train'][0]['labels']))

<class 'list'>
<class 'list'>


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=data_prepared_split["train"],
    eval_dataset=data_prepared_split["train"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.tokenizer,
)
# data_collator=data_collator,

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
processor.save_pretrained(training_args.output_dir)

[]

training

In [ ]:
trainer.train()

TypeError: is_floating_point(): argument 'input' (position 1) must be Tensor, not BatchEncoding

# Push to hub

In [ ]:
kwargs = {
    "dataset_tags": "thinhlpg/vivoice-distil",
    "dataset": "datavie",  # a 'pretty' name for the training dataset
    "language": "vie",
    "model_name": "Whisper Base Vie",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-base",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub(**kwargs)